# self

These tasks operate on this `jupyak` as a repo, not a package.

In [ ]:
import json
import os
import re
import shutil
from pathlib import Path

import doit
import importnb

import jupyak

with importnb.Notebook():
    from jupyak.tasks import _actions as A
    from jupyak.tasks import _form as F
    from jupyak.tasks import _graph as G
    from jupyak.tasks import _well_known as W
    from jupyak.tasks import _yak as Y
try:
    import tomllib
except ImportError:
    import tomli as tomllib

## tasks

> apply formatting to `jupyak`'s source files

In [ ]:
def task_fix():
    lintables = [*ALL_IPYNB, *ALL_PY]
    yield dict(
        name="py",
        doc="> clean up python",
        actions=[
            ["ruff", "--fix-only", *ALL_PY],
            ["ruff", "format", *ALL_PY],
        ],
        file_dep=[*ALL_PY, PPT],
    )

    yield dict(
        name="ipynb",
        doc="> clean up notebooks",
        actions=[
            ["nbstripout", *ALL_IPYNB],
            ["ruff", "--fix-only", *ALL_IPYNB],
            ["ruff", "format", *ALL_IPYNB],
        ],
        file_dep=[*ALL_IPYNB, PPT],
    )

    yield dict(
        name="pretty",
        doc="> clean up web-adjacent files with prettier",
        actions=[[*PRETTIER, *PRETTIER_ARGS, *ALL_PRETTIER]],
        file_dep=[*ALL_PRETTIER, JS_STATE],
    )

In [ ]:
def task_env():
    env_deps = {BINDER: GH, GH: DOCS}
    env_yml = "environment.yml"
    for dest, src in env_deps.items():
        yield dict(
            name=f"{dest.name}:yml",
            doc=f"> fix the {dest.name} env (from {src.name})",
            actions=[
                (
                    A.replace_between,
                    [src / env_yml, dest / env_yml, f"### {src.name}/{env_yml} ###"],
                )
            ],
            targets=[dest / env_yml],
            file_dep=[src / env_yml],
        )

    if not SKIP_CONDA_LOCK:
        lock = [f"{CONDA_EXE}-lock"]
        for env_dir in [BINDER, DOCS, GH]:
            yml_path = env_dir / env_yml
            yield dict(
                name=f"{env_dir.name}:lock",
                actions=[
                    A.AnnouncingCmdAction(
                        [*lock, "--mamba", "--file", env_yml, "--kind", "explicit"],
                        cwd=str(env_dir),
                        shell=False,
                    ),
                ],
                targets=[env_dir / BINDER_LOCK.name],
                file_dep=[yml_path],
            )

> build python distributions of `jupyak`

In [ ]:
def task_js():
    yarn_dep = [JS_PACKAGE, JS_PACKAGE_ROOT, JS_RC]
    if JS_LOCK.exists():
        yarn_dep += [JS_LOCK]
    yield dict(
        name="install",
        doc="> install nodejs dependencies",
        actions=[YARN],
        file_dep=yarn_dep,
        targets=[JS_STATE],
    )

In [ ]:
def task_build():
    yield dict(
        name="flit",
        doc="> build python distributions",
        actions=[
            ["pyproject-build", ".", "--no-isolation"],
            (A.sha256_some, [SHA256SUMS, DIST, ["*.whl", "*.tar.gz"]]),
        ],
        file_dep=[*ALL_SRC_PY, PPT, *ALL_IPYNB, LICENSE, README],
        targets=[WHEEL, SDIST],
    )

In [ ]:
def task_test():
    yield dict(
        name="pytest",
        doc="> test with pytest",
        actions=[["pytest", "tests"]],
        file_dep=[*ALL_SRC_PY, *ALL_TEST_PY, *ALL_IPYNB, PPT],
        targets=[REPORT_PYTEST, REPORT_HTMLCOV],
    )

> build the documentation site for `jupyak` itself

In [ ]:
def task_docs():
    sphinx_deps = [
        *ALL_DOCS_STATIC,
        *ALL_SRC_PY,
        *ALL_DOCS_PY,
        *ALL_DOCS_MD,
        *ALL_IPYNB,
        *ALL_DOCS_TEMPLATES,
        DOCS_GRAPH,
        PPT,
        README,
        LICENSE,
        CONTRIBUTING,
    ]
    sphinx_task_dep = []
    for svg in ALL_DOCS_SVG:
        tmp = BUILD / "scour" / svg.name
        yield dict(
            name=f"scour:{svg.name}",
            actions=[
                (doit.tools.create_folder, [tmp.parent]),
                [*SCOUR_ARGS, svg, tmp],
                (A.copy, [svg, tmp]),
                (A.clean, [tmp]),
            ],
            file_dep=[svg],
        )

    if shutil.which("convert") and shutil.which("inkscape"):
        yield dict(
            name="favicon",
            doc="rebuild favicons from svg source, requires imagemagick",
            file_dep=[DOCS_ICON],
            targets=[DOCS_FAVICON],
            actions=[
                A.run(
                    [
                        "convert",
                        "-verbose",
                        "-density",
                        "256x256",
                        "-background",
                        "transparent",
                        DOCS_ICON,
                        "-define",
                        "icon:auto-resize",
                        "-colors",
                        "256",
                        DOCS_FAVICON,
                    ],
                    {"cwd": ROOT},
                )
            ],
        )

    if WORK_DIST.exists():
        sphinx_task_dep += ["self:docs:deploy:work", "self:docs:deploy:favicon"]
        yield dict(
            name="deploy:work",
            doc="> copy built assets to docs static",
            actions=[(A.copy, [WORK_DIST, DOCS_STATIC_WORK])],
        )
        yield dict(
            name="deploy:favicon",
            doc="> copy docs favicon to lite",
            task_dep=["self:docs:deploy:work"],
            actions=[(A.copy, [DOCS_FAVICON, DOCS_STATIC_WORK_FAVICON])],
            file_dep=[DOCS_FAVICON],
            targets=[DOCS_STATIC_WORK_FAVICON],
        )

    yield dict(
        name="schema",
        doc="> generate schema for ``yak``",
        actions=[(Y.write_schema, [DOCS_SCHEMA, Y.Yak])],
        file_dep=[*ALL_IPYNB],
        targets=[DOCS_SCHEMA],
    )

    yield dict(
        name="form",
        doc="> build a form for the config",
        actions=[(F.write_form, [DOCS_FORM])],
        file_dep=[*ALL_IPYNB],
        targets=[DOCS_FORM],
    )

    yield dict(
        name="graph",
        doc="> build a graph of the tasks",
        actions=[(G.write_graph, [DOCS_GRAPH])],
        file_dep=[*ALL_IPYNB],
        targets=[DOCS_GRAPH],
    )

    yield dict(
        name="sphinx",
        doc="> build documentation with sphinx",
        actions=[["sphinx-build", "-b", "html", "docs", "build/docs"]],
        file_dep=sphinx_deps,
        task_dep=sphinx_task_dep,
        targets=[DOCS_BUILDINFO],
    )

> validate examples

## env vars

In [ ]:
CONDA_EXE = os.environ.get("CONDA_EXE", "conda")
SKIP_CONDA_LOCK = bool(json.loads(os.environ.get("JPYK_SKIP_CONDA_LOCK", "1")))

## commands

In [ ]:
SCOUR_ARGS = ["scour", "--enable-id-stripping", "--enable-comment-stripping"]
YARN = ["yarn"]
PRETTIER = [*YARN, "prettier"]
PRETTIER_ARGS = [
    "--cache",
    "--cache-location",
    "build/.cache/prettier",
    "--list-different",
    "--write",
]

## paths

In [ ]:
SRC = Path(jupyak.__file__).parent
TASKS = SRC / "tasks"

In [ ]:
PPT = (
    Path(W.PYPROJECT_TOML)
    if Path(W.PYPROJECT_TOML).exists()
    else SRC.parent.parent / W.PYPROJECT_TOML
)

In [ ]:
ROOT = PPT.parent.resolve()
TESTS = ROOT / "tests"
GH = ROOT / ".github"

In [ ]:
ALL_TEST_PY = sorted(p for p in TESTS.rglob("*.py") if not re.search(W.IPYNBC, str(p)))
ALL_SRC_PY = sorted(p for p in SRC.rglob("*.py") if not re.search(W.IPYNBC, str(p)))

## project data

In [ ]:
PPT_DATA = tomllib.loads(PPT.read_text(encoding="utf-8"))
PJ = PPT_DATA["project"]

## top-level

In [ ]:
BUILD = ROOT / "build"
DIST = ROOT / "dist"
README = ROOT / "README.md"
LICENSE = ROOT / "LICENSE"
CONTRIBUTING = ROOT / "CONTRIBUTING.md"

## binder

In [ ]:
BINDER = ROOT / ".binder"
BINDER_ENV = BINDER / W.ENVIRONMENT_YML
BINDER_LOCK = BINDER / "conda-linux-64.lock"

## tests

In [ ]:
REPORTS = BUILD / "reports"
REPORT_PYTEST = REPORTS / "pytest.html"
REPORT_HTMLCOV = REPORTS / "htmlcov/index.html"

## docs

In [ ]:
DOCS = ROOT / "docs"
DOCS_ENV = DOCS / W.ENVIRONMENT_YML
DOCS_STATIC = DOCS / "_static"
DOCS_IMG = DOCS_STATIC / "img"
DOCS_BUILD = BUILD / "docs"
DOCS_BUILDINFO = DOCS_BUILD / ".buildinfo"
DOCS_FORM = DOCS / "new.md"
DOCS_ICON = DOCS_IMG / "logo.svg"
DOCS_SCHEMA = DOCS_STATIC / "jupyak-v0.schema.json"
DOCS_FAVICON = DOCS_IMG / "favicon.ico"
DOCS_GRAPH = DOCS / "graph.md"
DOCS_TEMPLATES = DOCS / "_templates"
DOCS_CSS = DOCS_STATIC / "css"
DOCS_STATIC_WORK = DOCS_STATIC / "work"
DOCS_STATIC_WORK_FAVICON = DOCS_STATIC_WORK / "lite/lab" / DOCS_FAVICON.name
ALL_DOCS_STATIC = [
    p
    for p in [*DOCS_CSS.rglob("*.*"), *DOCS_IMG.rglob("*.*")]
    if "ipynb_checkpoints" not in str(p)
]
ALL_DOCS_PY = sorted(DOCS.rglob("*.py"))
ALL_DOCS_SVG = sorted(DOCS_IMG.glob("*.svg"))
ALL_DOCS_MD = [DOCS_GRAPH, *sorted(DOCS.rglob("*.md"))]
ALL_DOCS_TEMPLATES = [*DOCS_TEMPLATES.glob("*.html")]

# js

In [ ]:
JS = ROOT / "js"
JS_LOCK = ROOT / W.YARN_LOCK
JS_PACKAGE_ROOT = ROOT / W.PACKAGE_JSON
JS_PACKAGE = JS / W.PACKAGE_JSON
JS_STATE = ROOT / W.YARN_STATE
JS_RC = ROOT / W.YARNRC
JS_SRC = JS / "src"
ALL_TS = [*JS_SRC.rglob("*.ts"), *JS_SRC.rglob("*.tsx")]

## work

In [ ]:
WORK_PATH = Path(os.environ.get(W.ENV_VAR_WORK_DIR) or ROOT / "work")
WORK_DIST = WORK_PATH / "dist"
WORK_DIST_LITE_SHASUMS = WORK_DIST / "lite" / W.SHA256SUMS

## linting

In [ ]:
ALL_IPYNB = sorted([*TASKS.glob("*.ipynb"), *DOCS.glob("*.ipynb")])
ALL_PY = sorted([*ROOT.glob("*.py"), *ALL_SRC_PY, *ALL_DOCS_PY, *ALL_TEST_PY])
ALL_PRETTIER = [*ALL_TS, README]

## distributions

In [ ]:
WHEEL = DIST / f"""{PJ["name"]}-{PJ["version"]}{W.NOARCH_WHL}"""
SDIST = DIST / f"""{PJ["name"]}-{PJ["version"]}{W.SDIST_EXT}"""
SHA256SUMS = DIST / W.SHA256SUMS